In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-rsyiwjxw
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-rsyiwjxw
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=31d2b4e44ec15c279fd25dfa2d714bc2d12cbbfe0d53f6f2b59834cc4db7a4db
  Stored in directory: /tmp/pip-ephem-wheel-cache-zq5qjw89/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
     |████████████████████████████████| 1.0MB 3.9MB/s eta 0:00:01


## NER TASK 3 MODELS

In [12]:
import json
import pandas as pd
import numpy as np
import re
import string
from keras.models import Model, Input,Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from keras_contrib.layers import CRF

#### F1 MACRO

In [13]:
from tensorflow.keras.callbacks import Callback
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.utils import flatten
from sklearn.metrics import f1_score

def F1_Score(y_test,y_pred):
    y_test = np.argmax(y_test, axis=2)
    # Flatten y_pred if necessary
    if y_pred.ndim == 3:
        y_pred = np.argmax(y_pred, axis=2)
    classes = np.unique(y_test)
    precision = []
    recall = []
    f1 = []
    for c in classes:
        tp = np.sum((y_test == c) & (y_pred == c))
        fp = np.sum((y_test != c) & (y_pred == c))
        fn = np.sum((y_test == c) & (y_pred != c))
        
        precision_c = tp / (tp + fp) if tp + fp > 0 else 0
        recall_c = tp / (tp + fn) if tp + fn > 0 else 0
        f1_c = 2 * (precision_c * recall_c) / (precision_c + recall_c) if precision_c + recall_c > 0 else 0
        
        precision.append(precision_c)
        recall.append(recall_c)
        f1.append(f1_c)

    macro_precision = np.mean(precision)
    macro_recall = np.mean(recall)
    macro_f1 = np.mean(f1)

    return macro_f1

class F1ScoreCallback(Callback):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs={}):
        # Predict on training and validation sets
        y_train_pred = np.argmax(self.model.predict(self.X_train), axis=-1)
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=-1)

        # Calculate F1 scores for training and validation sets
        f1_train = F1_Score(self.y_train, y_train_pred)
        f1_val = F1_Score(self.y_val, y_val_pred)

        if logs is not None:
            logs['f1_score_train'] = f1_train
            logs['f1_score_val'] = f1_val

        # Print F1 scores
        print(f' - f1_train: {f1_train:.4f} - f1_val: {f1_val:.4f}')
f1_callback = F1ScoreCallback(X_train, np.array(y_train), X_val, np.array(y_val))

In [14]:
# LOAD MODEL
import joblib
X_test,y_test = joblib.load('/kaggle/input/test-data-ner/test_data.pkl')


#### GLOVE

In [16]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ner-models/glove_ner_3.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x78331a743668>>
949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.81      0.63      0.71       177
       B_COURT       0.18      0.64      0.29        11
        B_DATE       0.40      0.46      0.43       310
         B_GPE       0.60      0.67      0.64       121
       B_JUDGE       0.89      0.90      0.90       383
         B_ORG       0.58      0.32      0.41        47
B_OTHER_PERSON       0.72      0.22      0.34        58
  B_PETITIONER       0.88      0.86      0.87       326
   B_PRECEDENT       0.57      0.84      0.68       195
   B_PROVISION       0.71      0.93      0.80       344
  B_RESPONDENT       0.73      0.66      0.69       276
     B_STATUTE       0.95      0.88      0.92       258
     B_WITNESS       0.84      0.93      0.88       102
 I_CASE_NUMBER       0.95      0.75      0.84      1793
    

#### WORD 2 VEC

In [18]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ner-models/word2vec_ner_3.h5')

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.74      0.66      0.70       177
       B_COURT       0.36      0.36      0.36        11
        B_DATE       0.69      0.52      0.59       310
         B_GPE       0.62      0.57      0.59       121
       B_JUDGE       0.83      0.85      0.84       383
         B_ORG       0.34      0.53      0.42        47
B_OTHER_PERSON       0.64      0.31      0.42        58
  B_PETITIONER       0.77      0.88      0.82       326
   B_PRECEDENT       0.66      0.76      0.71       195
   B_PROVISION       0.62      0.78      0.69       344
  B_RESPONDENT       0.77      0.67      0.71       276
     B_STATUTE       0.91      0.88      0.89       258
     B_WITNESS       0.70      0.92      0.80       102
 I_CASE_NUMBER       0.91      0.84      0.87      1793
       I_COURT       0.99      0.99      0.99     66228
        I_DATE       0.00      0.00      0.0

#### Fast Text

In [19]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ner-models/fasttext_ner_3.h5')

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


949
949/949 [==============================] - 2s 2ms/step
                precision    recall  f1-score   support

 B_CASE_NUMBER       0.55      0.43      0.48       177
       B_COURT       0.21      0.27      0.24        11
        B_DATE       0.46      0.37      0.41       310
         B_GPE       0.50      0.42      0.46       121
       B_JUDGE       0.73      0.86      0.79       383
         B_ORG       0.31      0.09      0.13        47
B_OTHER_PERSON       0.19      0.21      0.20        58
  B_PETITIONER       0.77      0.80      0.78       326
   B_PRECEDENT       0.32      0.75      0.45       195
   B_PROVISION       0.53      0.64      0.58       344
  B_RESPONDENT       0.30      0.64      0.41       276
     B_STATUTE       0.90      0.85      0.87       258
     B_WITNESS       0.45      0.87      0.60       102
 I_CASE_NUMBER       0.91      0.57      0.70      1793
       I_COURT       0.99      0.98      0.99     66228
        I_DATE       0.00      0.00      0.0

## ATE TASK 3 MODELS

In [20]:
## Load data 
import joblib
X_test,y_test = joblib.load('/kaggle/input/test-data-ate/test_data_ate.pkl') 

#### Glove

In [21]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ate-models/glove_ate_3.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7832c7b73e48>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

      B_DATE       0.70      0.77      0.73       463
     B_JUDGE       0.99      0.99      0.99     24878
I_PETITIONER       0.67      0.53      0.59       243

    accuracy                           0.98     25584
   macro avg       0.79      0.76      0.77     25584
weighted avg       0.98      0.98      0.98     25584



#### Word2V

In [22]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ate-models/word2vec_ate_3.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x783318848ac8>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

      B_DATE       0.64      0.74      0.68       463
     B_JUDGE       0.99      0.99      0.99     24878
I_PETITIONER       0.51      0.55      0.53       243

    accuracy                           0.98     25584
   macro avg       0.71      0.76      0.74     25584
weighted avg       0.98      0.98      0.98     25584



#### Fast Text

In [23]:
from tensorflow.keras.models import load_model
from keras.models import load_model

def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "crf_loss": loss, "crf_viterbi_accuracy":accuracy}

def load_keras_model(path):
    model = load_model(path, custom_objects=create_custom_objects())
    return model

model = load_keras_model('/kaggle/input/ate-models/fasttext_ate_3.h5')
print(model.summary)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
print(len(X_test))
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)

/opt/conda/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x783319613240>>
328
328/328 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

      B_DATE       0.73      0.64      0.68       463
     B_JUDGE       0.99      0.99      0.99     24878
I_PETITIONER       0.55      0.52      0.54       243

    accuracy                           0.98     25584
   macro avg       0.76      0.72      0.74     25584
weighted avg       0.98      0.98      0.98     25584

